# Test-Charaktere
Für erste Tests erst einmal 2 simple Charaktere um die Grundlagen zu testen.

In [9]:
# Modul um Würfel zu simulieren
import random

In [10]:
def w20():
    """
    Simple Simulation eines W20 Würfels.
    Benötigt keinen Input und gibt immer einen Wert zwischen 1 und 20 zurück.
    """

    return random.randint(1, 20)


def w6():
    """
    Simple Simulation eines W20 Würfels.
    Benötigt keinen Input und gibt immer einen Wert zwischen 1 und 6 zurück.
    """

    return random.randint(1, 6)

## Glossar

In [11]:
# Ein kurzes Glossar ist nötig da es einige Abkürzungen gibt
glossar_one = {
    "MU": "Mut",
    "KL": "Klugheit",
    "IN": "Intuition",
    "CH": "Charisma",
    "FF": "Fingerfertigkeit",
    "GE": "Gewandheit",
    "KO": "Konstitution",
    "KK": "Körperkraft",

    "LP": "Lebenspunkt",
    "AsP": "Astralpunkt",
    "AUS": "Ausdauer",
    "INIT": "Initiative",
    "AP": "Abenteurerpunkte"
}

# Da auch die umgekehrte Richtung abgefragt werden kann wird das Glossar einmal gedreht...
glossar_two = {val: key for key, val in glossar_one.items()}

# ... um beide "Hälften" dann zu einem zusammenzufügen
# Hierbei wird der built-in Operator "|" benutzt
glossar = glossar_one | glossar_two

# Liste der vorhandenen Eigenschaften welche jeder Charakter besitzt
# Wir nutzen eine Liste, damit über die Einträgt iteriert werden kann
dsa3_eigenschaften = ["MU", "KL", "IN", "CH", "FF", "GE", "KO", "KK"]

# 
dsa3_fertigkeiten = {
    "Tierkunde": [["KL", "KL", "IN"], w6()]
}

# 
dsa3_startwerte = {
    "LP": 12,
    "AsP": 12,
    "AUS": 12,
    "INIT": 12,
    "AP": 0
}

In [12]:
def generate_eigenschaften(skills):
    """
    Generiert ein dictionary an random Werten (1-20) für die eigenschaften eines
    DSA Charakters. Diese dienen erst einmal nur Tetszwecken und können später
    noch verfeinert werden.
    """

    eigenschaften = {}

    for skill in skills:
        eigenschaften[skill] = w20()

    return eigenschaften


def generate_fertigkeiten(skills):
    """
    
    """

    fertigkeiten = {}

    for skill in skills:
        fertigkeiten[skill] = [w20()]



def generate_charakterbogen(name, alter, geschlecht, rasse, heldentyp):
    """

    """

    label = ["name", "alter", "geschlecht", "rasse", "heldentyp"]
    infos = [name, alter, geschlecht, rasse, heldentyp]

    charakterbogen = dict(zip(label, infos)) | dsa3_startwerte
    
    charakterbogen["eigenschaften"] = generate_eigenschaften(dsa3_eigenschaften)
    charakterbogen["fertigkeiten"] = generate_fertigkeiten(dsa3_fertigkeiten)

    return charakterbogen


In [13]:
data_parnileo_parnilei = generate_charakterbogen("Parnileo Parnilei", 461, "männlich", "Mensch", "Magier")
data_edgar_allan_parn = generate_charakterbogen("Edgar Allan Parn", 216, "männlich", "Mensch", "Magier")

In [ ]:
class Held():
    def __init__(self, file):
        if file is not None:
            for key, value in file.items():
                setattr(self, key, value)


    def eigenschaftsprobe(self, probe, bias=0):
        wurf = w20()

        if wurf > self.eigenschaften[glossar[probe]] + bias:
            print(f"Eigenschaftsprobe misslungen! Der nötige {probe}-Wert von {self.eigenschaften[glossar[probe]]} wurde durch {wurf} nicht unterworfen!")
        
        else:
            print(f"Eigenschaftsprobe geschafft! Der nötige {probe}-Wert von {self.eigenschaften[glossar[probe]]} wurde durch {wurf} unterworfen!")
    

    def fertigkeitsprobe(self, probe, bias=0):
        """
        
        """

        ausgleich = self.fertigkeiten[probe][1]

        for fertigkeit in self.fertigkeiten[probe]:
            wurf = w20()


In [ ]:
parnileo_parnilei = Held(data_parnileo_parnilei)
edgar_allan_parn = Held(data_edgar_allan_parn)

glossar["Klugheit"]

'KL'

In [16]:
print(f"Hallo, ich bin {parnileo_parnilei.name} und ich habe in Mut einen Wert von {parnileo_parnilei.eigenschaften['MU']}.")

parnileo_parnilei.eigenschaftsprobe("Mut")

Hallo, ich bin Parnileo Parnilei und ich habe in Mut einen Wert von 15.
Eigenschaftsprobe misslungen! Der nötige Mut-Wert von 15 wurde durch 16 nicht unterworfen!
